### Take in the privacy stories from an application and use the LLM to determine appropriate solutions. 



In [1]:
import pandas as pd

from conf import data_dir

#################################################
# Manually gathered privacy stories for example #
#################################################


privacy_stories = [
    """1. **We collect Personal Data and Usage Data** to enhance operational efficiency and user experience.
   - **Purpose:** To provide and maintain the service, including monitoring the usage of our service. This assists in improving service functionalities and ensuring a smoother user experience.""",

    """2. **We use Personal Data** for account management and service delivery.
   - **Purpose:** To manage user registrations and provide access to various functionalities of the service that are available to registered users. This data is also used to perform contracts by ensuring the delivery of products, items, or services as agreed.""",

    """3. **We use Personal Data** for communication purposes.
   - **Purpose:** To contact users via email, telephone calls, SMS, or other electronic communications regarding updates, security information, or promotional communications related to functionalities, products, or contracted services.""",

    """4. **We use Personal Data** for promotional and analytical purposes.
   - **Purpose:** To provide users with news, special offers, and general information about similar goods, services, and events which might interest them, and also for data analysis to identify usage trends and evaluate the effectiveness of promotional campaigns.""",

    """5. **We share Personal Data with Service Providers and Affiliates** to enhance service delivery.
   - **Purpose:** To monitor and analyze the usage of our service and to maintain service standards across different entities within our corporate family, ensuring consistent adherence to our privacy policy.""",

    """6. **We share Personal Data** in the context of business transactions.
   - **Purpose:** In the event of mergers, sales of company assets, or acquisition, personal data may be transferred as part of the assets, facilitating business continuity or integration into new corporate structures.""",

    """1. **We collect Personal Data** (such as Usage Data including IP addresses, browser types, visit timings, and other diagnostic data) **for the purpose of Functionality** to provide and maintain our service, manage user accounts, and optimize service delivery.""",

    """2. **We use Personal Data** to **manage your account** for the purpose of Functionality, enabling access to different functionalities of the service for registered users.""",

    """3. **We contact You** using Personal Data (via email, telephone calls, SMS, or other forms of electronic communication) **for the purpose of Contact**, to provide updates and communications about functionalities, products, or contracted services, and to inform about necessary security updates.""",

    """4. **We share Personal Data with Service Providers** for the purpose of Analytics and Functionality, to monitor and analyze service usage and assist in the operation of our service.""",

    """5. **We use Personal Data** for **the purpose of Analytics**, to perform data analysis, identify usage trends, determine the effectiveness of promotional campaigns, and evaluate and improve our service, products, services, and marketing strategies.""",

    """6. **We use Personal Data** for business transfers such as mergers or acquisitions **for the purpose of Analytics**, considering it as among the assets transferred.""",

    """7. **We use Personal Data** to **provide you with news, special offers, and information** about goods, services, or events similar to those you have already purchased or inquired about **for the purpose of Advertisement**.""",

    """8. **We share Personal Data with Affiliates** who are required to honor this Privacy Policy **for the purpose of Functionality**, which supports integrated and efficient service offerings.""",

    """1. **We share Personal Data with service providers** for the purpose of analytics and internal analysis.""",

    """2. **We share Personal Data with affiliates** to ensure adherence to the same privacy policy across our controlled entities.""",

    """3. **We share Personal Data with business partners** to offer specific products, services, or promotions.""",

    """4. **We share Personal Data publicly when users interact in public areas** allowing it to be viewed or distributed outside for the purpose of public user interaction.""",

    """5. **We share Personal Data with explicit consent for any other purpose** where user consent has been explicitly given, ensuring respect for user's control over their information.""",

    """6. **We retain Personal Data only as long as necessary** for the purpose of policy compliance, to comply with our legal obligations, to resolve disputes, and to enforce our legal agreements and policies.""",

    """7. **We retain Usage Data for internal analysis** but generally for a shorter period unless required for security enhancements or to improve functionality, or when legally obligated."""
]

stories_list = "\n".join(privacy_stories)

"""
Gather stories from llm output files
"""

chosen_file = 'only_privacy_stories_1-5.csv'
df = pd.read_csv(f"{data_dir}/{chosen_file}")



In [ ]:
import json
import os 

from conf import data_dir

# Read the JSON file
with open(f'{data_dir}/privacy_patterns.json', 'r') as f:
    data = json.load(f)

# Initialize an empty list to store the problems
problems_list = []

# Iterate over each item in the JSON data and extract the context and problem entries
for pattern_url, item in data.items():
    context = item.get('context', '')
    problem = item.get('problem', '')
    pattern_name = pattern_url.split('/')[-1]  # Extract the pattern name from the URL
    problems_list.append({
        'pattern': pattern_name,
        'context': context,
        'problem': problem
    })

with open(f'{data_dir}/pbd_text/pbd_wiki.txt', encoding='latin-1') as f:
    pbd_wiki = f.read()

pbd_booklet = {}
for file_name in os.listdir(f'{data_dir}/pbd_text/pbd_booklet'):
    with open(f'{data_dir}/pbd_text/pbd_booklet/{file_name}', encoding='latin-1') as f:
        pbd_booklet[file_name] = f.read()

print(pbd_booklet)



In [ ]:
# Generate initialization string 
build_string = "You are a helpful AI assistant"
# Generate the prompt
prompt = ( 
    f"Now read through this list of stories:\n\n{stories_list}\n\n"
    "Identify which of the following strategies the system described"
    "by these stories which define there practices of private data, "
    "could be adapted to improve there system with explanation. At the"
    "end write a list of what of the 9 strategies are most needed in json"
)

prompt += pbd_wiki

def add_pbd_stuff(pbd_stuff, prompt):
    for file_name, content in pbd_stuff.items():
        prompt += content
    return prompt


# Add each problem entry to the prompt
def add_ppatterns(prompt):
    prompt += "Identify which of the following patterns are related to the privacy stories:\n"
    for problem in problems_list:
        prompt += f"Pattern : ['{problem['pattern']}'] - Context: {problem['context']} : Problem: {problem['problem']}\n"
    # Add output instructions 
    prompt += (
        "Provide your answers in a list of patterns with the format: {pattern} is related with these {selected stories related to that pattern} because"
    )
    return prompt



print(prompt)